In [2]:
import pandas as pd
import numpy as np
from sympy import *

#### Задание 1.1
Пусть прибыль в вашей компании выражается следующей функцией, которая зависит от параметра $x$  — количества производимых товаров:
$$f(x)=-x^{4}+6 x^{3}-4 x^{2}+80$$
Найдите максимально возможную прибыль, которую вы можете получить, варьируя количество произведённых товаров. 

In [9]:
x = symbols('x')
f = -x**4 + 6*x**3 -4*x**2 + 80
# f_x = -4*x**3 + 18*x**2 - 8*x
f_x = f.diff(x)
solve(f_x,x)

[0, 1/2, 4]

In [5]:
f_x.subs(x,-1),f_x.subs(x,0.25),f_x.subs(x,1),f_x.subs(x,5)

(30, -0.937500000000000, 6, -90)

In [11]:
f.subs(x,0),f.subs(x,4)

(80, 144)

#### Задание 1.4
Допустим, вы хотите произвести некоторое количество товара, которое зависит от часов работы двух ключевых сотрудников следующим образом:
$$f(x, y)=x^{2}+2 y^{2}$$
Однако вы можете оплатить этим сотрудникам не более 20 часов работы.

Какое наибольшее количество товаров вы сможете произвести в таком случае?

Помните, что количество часов работы должно быть целым, поэтому, прежде чем вычислять итоговый результат, округлите часы работы до целого.

In [14]:
# x,y,l1,l2,x_1,x_2 = symbols('x,y,l1,l2,x_1,x_2')
# f = x**2 + 2*y**2
# L = f + l1*(-x-y+x_1**2) + l2*(-x-y-x_2**2 + 20)

In [21]:
x,y,l1,x1 = symbols('x,y,l1,x1')
f = x**2 + 2*y**2
L = f + l1*(-x - y - x1**2 + 20)

In [22]:
L_x, L_y, L_l1, L_x1 = L.diff(x), L.diff(y), L.diff(l1), L.diff(x1)
L_x, L_y, L_l1, L_x1

(-l1 + 2*x, -l1 + 4*y, -x - x1**2 - y + 20, -2*l1*x1)

In [24]:
slv = solve([L_x, L_y, L_l1, L_x1],[x,y,l1,x1])
slv

[(0, 0, 0, -2*sqrt(5)), (0, 0, 0, 2*sqrt(5)), (40/3, 20/3, 80/3, 0)]

In [30]:
x_int, y_int = round(slv[2][0]),round(slv[2][1])
f.subs(x,x_int).subs(y,y_int)

267

#### Задание 2.7
Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии. Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки.

Загрузите стандартный датасет об алмазах из библиотеки Seaborn:

```python
import seaborn as sns
df = sns.load_dataset('diamonds')
```
Удалите часть признаков:
```python
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
```
Закодируйте категориальные признаки:
```python
df = pd.get_dummies(df, drop_first=True)
```
Логарифмируйте признаки:
```python
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])
```
Определите целевую переменную и предикторы:
```python
X_cols = [col for col in df.columns if col!='price'] 
X = df[X_cols]
y = df['price']
```
Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33).

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (функция `SGDRegressor`). Отберите с помощью `gridsearch()` оптимальные параметры по следующей сетке:
```python
'learning_rate': ['constant'],
    'eta0': np.logspace(-4,-1,4),
    'max_iter': np.logspace(0,3,10),
    'loss': ['squared_loss', 'epsilon_insensitive'],
    'penalty': ['elasticnet'], 
    'alpha': np.logspace(-3,3,15),
    'l1_ratio': np.linspace(0,1,11),
```

Обучите регрессию с оптимальными параметрами. В качестве ответа введите получившееся значение `MSE`, предварительно округлив его до третьего знака после точки-разделителя.

In [46]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

parameters = {
    'learning_rate': ['constant'],
    'eta0': np.logspace(-4,-1,4),
    'max_iter': np.logspace(0,3,10),
    'loss': ['squared_loss', 'epsilon_insensitive'],
    'penalty': ['elasticnet'], 
    'alpha': np.logspace(-3,3,15),
    'l1_ratio': np.linspace(0,1,11)
}

sgd = SGDRegressor(random_state=42, early_stopping = True)
sgd_cv = GridSearchCV(estimator=sgd, param_grid=parameters, n_jobs=-1)
sgd_cv.fit(X_train, y_train)

print(sgd_cv.best_params_)

sgd = SGDRegressor(**sgd_cv.best_params_, random_state = 42)

sgd.fit(X_train, y_train)
sgd.score(X_train, y_train)
ls = sgd.predict(X_test)
from sklearn.metrics import mean_squared_error
round(mean_squared_error(y_test, ls), 3)

{'alpha': 0.001, 'eta0': 0.01, 'l1_ratio': 0.9, 'learning_rate': 'constant', 'loss': 'squared_loss', 'max_iter': 10.0, 'penalty': 'elasticnet'}


0.045

#### Задание 3.1
Найдите третий корень полинома $f(x) = 6x^5 - 5x^4 - 4x^3 + 3x^2$, взяв за точку старта 0.7. Введите получившееся значение с точностью до трёх знаков после точки-разделителя.

Попробуйте реализовать алгоритм с использованием Python на основе алгоритма градиентного спуска, изученного в предыдущем модуле.

In [53]:
def func(x):
    return 6*x**5-5*x**4-4*x**3+3*x**2
 
def func_dx(x):
    return 30*x**4-20*x**3-12*x**2+6*x

iter_count = 0
epsilon = 0.000001
x_curr = 0.7
 
while (abs(func(x_curr)) > epsilon):
    x_curr -= func(x_curr)/func_dx(x_curr)
    iter_count += 1
#     print(x_curr)
print(round(x_curr,3))    

0.629


#### Задание 3.6
Дана функция $f(x) = x^3 - 72x - 220$. Найдите решение уравнения $f(x) = 0$  для поиска корня в окрестностях точки $x_0 = 12$. Ответ округлите до трёх знаков после точки-разделителя.

In [61]:
func = lambda x: x**3 - 72*x - 220
func_dx = lambda x: 3*x**2 - 72

In [65]:
iter_count = 0
epsilon = 0.000001
x_curr = 12
 
while (abs(func(x_curr)) > epsilon):
    x_curr -= func(x_curr)/func_dx(x_curr)
    iter_count += 1
#     print(x_curr)
print(iter_count,round(x_curr,3)) 

4 9.727


In [58]:
func(x_curr),func(919/90)

9.993225376092596e-07

#### Задание 3.7
Найдите положительный корень для уравнения $x^2 + 9x - 5 = 0$.

В качестве стартовой точки возьмите $x_0 = 2.2$.

Расчёт произведите поэтапно или с помощью Python.

Ответ округлите до двух знаков после точки-разделителя.

In [66]:
func = lambda x: x**2 + 9*x - 5
func_dx = lambda x: 2*x + 9


In [69]:
epsilon = 0.00001
iter_count = 0
x_curr = 2.2
while func(x_curr) > epsilon:
    x_curr -= func(x_curr)/func_dx(x_curr)
    iter_count += 1
print (iter_count, round(x_curr,2))

4 0.52


#### Задание 3.9
С помощью метода Ньютона найдите точку минимума для функции $f(x) = 8x^3 - 2x^2 - 450$.

Для расчётов используйте Python.

В качестве стартовой точки возьмите $42$, точность примите за $0.0001$.

Ответ округлите до трёх знаков после точки-разделителя.

In [74]:
func = lambda x: 8*x**3 - 2*x**2 -450
func_dx = lambda x: 24*x**2 - 4*x
func_dxx = lambda x: 48*x - 4

epsilon = 0.0001
x_curr = 42.
iter_count = 0

while func_dx(x_curr)>epsilon:
    x_curr -= func_dx(x_curr)/func_dxx(x_curr)
    iter_count += 1
    
print(iter_count,round(x_curr,3))

12 0.167


Давайте рассмотрим, как с помощью функций Python мы сможем применить квазиньютоновские методы для оптимизации функции $f(x,y) = x^2 + y^2$.

Подгрузим необходимые библиотеки:

In [75]:
import numpy as np
from scipy.optimize import minimize

Определим функцию, которую будем оптимизировать и определим градиент для функции.

In [80]:
func = lambda x: x[0]**2.0 + x[1]**2.0
grad_func = lambda x: np.array([x[0] * 2, x[1] * 2])

x_0 = [1.0, 1.0]

result = minimize(func, x_0, method='BFGS', jac=grad_func)

print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Можно повторить то же самое с вариацией  `L-BFGS-B`:

In [81]:
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Итак, мы обсудили один из самых эффективных на сегодняшний день алгоритмов — вариацию BFGS квазиньютоновских методов. Вы будете регулярно сталкиваться с этим алгоритмом при решении различных задач и при использовании библиотек для оптимизации. Так что давайте попрактикуемся: в этом юните мы посмотрели фрагмент поэтапного разбора метода BFGS для функции $f(x,y) = x^2 - xy + y^2 + 9x - 6y + 20$  — давайте завершим начатое и найдём точку минимума

#### Задание 4.1
Найдите точку минимума для функции $f(x,y) = x^2 - xy + y^2 + 9x - 6y + 20$.

В качестве стартовой возьмите точку $(-400, -400)$.

Значения координат округлите до целого числа.

In [86]:
func = lambda x: x[0] ** 4.0 - x[0] * x[1] + x[1] ** 2 + 9 * x[0] - 6 * x[1] + 20
grad_func = lambda x: np.array([2 * x[0] - x[1] + 9, -x[0] + 2 * x[1] - 6])

x_0 = [-400, -400]
result = minimize(func, x_0, method='BFGS', jac=grad_func)
solution = result['x']
print (solution.round())

[-4.  1.]


#### Задание 4.4
Найдите минимум функции  $f(x) = x**2 - 3x + 45$  с помощью квазиньютоновского метода `BFGS`.

В качестве стартовой точки возьмите $x = 10$.

В качестве ответа введите минимальное значение функции в достигнутой точке.

In [94]:
func = lambda x: x**2 - 3*x + 45
func_dx = lambda x: 2*x - 3
x_0 = 10
result = minimize(func,x_0,method='BFGS',jac=func_dx)
result.x

array([1.5])

#### Задание 4.5
Решите предыдущую задачу, применяя модификацию `L-BFGS-B`.

В каком случае получилось меньше итераций?

In [93]:
result_l = minimize(func,x_0,method='L-BFGS-B',jac=func_dx)
result_l.nit, result.nit

(2, 4)

#### Задание 4.7
Найдите минимум функции $f(x,y) = x^4 + 6*y^2 + 10$, взяв за стартовую точку $(100,100)$.
Какой алгоритм сошелся быстрее?

In [98]:
func = lambda x: x[0]**4 + 6*x[1] + 10
grad_func = lambda x: np.array([4*x[0]**3 ,12*x[1]])
x_0 = [100,100]
res_bfgs = minimize(func,x_0,method='BFGS',jac=grad_func)
res_lbfgsb = minimize(func,x_0,method='L-BFGS-B',jac=grad_func)
res_bfgs.nit,res_lbfgsb.nit

(34, 15)

Каково минимальное значение функции в найденной точке минимума?

In [107]:
res_bfgs.x , res_bfgs.message, func(res_bfgs.x)

(array([1.23319058e-02, 5.70395858e-14]),
 'Optimization terminated successfully.',
 10.000000023127422)

In [106]:
res_lbfgsb.x, res_lbfgsb.message,  func(res_lbfgsb.x)

(array([ -1.65591126, -39.95629811]),
 'ABNORMAL_TERMINATION_IN_LNSRCH',
 -222.21899385101636)

#### Задание 5.5
Магазин спортивных товаров получает прибыль в размере 6 долларов с каждого проданного футбольного мяча и 5.5 долларов — с бейсбольного.

Каждый месяц магазин продаёт от 35 до 45 футбольных мячей и от 40 до 55 бейсбольных.

Известно, что в этом месяце у магазина есть в общей сложности 80 мячей.

Какую максимальную прибыль в этом месяце может получить магазин от продажи мячей?

#### Пример № 1. SciPy (scipy.optimize.linprog)

У нас есть 6 товаров с заданными ценами на них и заданной массой.

Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.

Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [110]:
# Создадим переменные на основе предложенных данных:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

Сформулируем задачу линейного программирования. Максимизируем произведение стоимости на количество, учитывая, что произведение веса на искомое количество товаров должно укладываться во вместимость сумки:
$$\max \sum v_{i} x_{i}$$
$$\sum w_{i} x_{i} \leq C$$
Из предыдущего юнита мы уже знаем, что в векторно-матричной форме наша задача должна формулироваться в следующем виде:
$$\operatorname{min} c^{T} x$$
$$A x \leq b$$
Получается, что в наших обозначениях мы имеем следующее:
$$c=-v, \ A=w^{T}, \ b=(C)$$
Здесь нам необходимо вспомнить линейную алгебру, так как очень важно, чтобы векторы были в нужных нам размерностях, иначе мы не сможем использовать матричное умножение. Вектор  размера  мы превращаем в матрицу размера  с помощью функции expand_dims(). Создаём все необходимые переменные:

In [112]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

# Передаём подготовленные переменные в оптимизатор SciPy:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.50000000003076
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24940067e-11])
  status: 0
 success: True
       x: array([6.18738531e-14, 1.05853305e-12, 1.21475943e-13, 7.50000000e+00,
       4.00246691e-13, 4.71394161e-13])

#### Пример № 2. CVXPY

Снова решим задачу из примера № 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.

SciPy не умеет решать такие задачи, поэтому будем использовать новую библиотеку CVXPY.

In [130]:
import cvxpy
import cvxopt
# С помощью CVXPY создадим переменную-массив. Укажем его размерность, а также условие, что все числа в массиве должны быть целыми:
x = cvxpy.Variable(shape=n, integer = True)

# Далее зададим ограничения, используя матричное умножение:
constraint = (A @ x <= b)
total_value = c @ x

# Переходим непосредственно к решению задачи:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])
# problem = cvxopt.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

# Вызываем получившееся решение:
problem.solve(solver='ECOS_BB')
# problem.solve(solver='GLPK')
# problem.solve()

-138412039.00000018

В результате получаем бесконечность. Это совершенно нереалистично.

В таком случае будем рассматривать только положительные значения $x$: $x \geq 0$

В переформулированном виде задача будет решаться следующим образом:

In [131]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c @ x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
# problem.solve()
problem.solve(solver='ECOS_BB')
x.value

C:\Users\Manych\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


array([7.01265807e-10, 7.99333027e-10, 3.58703130e-10, 7.00000000e+00,
       4.67143021e-10, 9.34955115e-10])

А что если мы можем брать не любое количество товаров, а только один или не брать их вовсе? Задаём $x$ типа boolean.

In [133]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c @ x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve(solver='ECOS_BB')
# problem.solve()
#-17.0
x.value
#array([1., 0., 0., 1., 0., 1.])

array([ 1.00000000e+00,  8.03480068e-11, -2.09342022e-11,  1.00000000e+00,
        1.08260122e-10,  1.00000000e+00])

#### Пример № 3. PuLP

В нашей каршеринговой компании две модели автомобилей: модель $A$ и модель $B$. Автомобиль $A$ даёт прибыль в размере 20 тысяч в месяц, а автомобиль  $B$ — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

- Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа $A$, и 5 дней — для каждого автомобиля типа  $B$.
- Заводу требуется 3 дня, чтобы изготовить модель $A$, и 6 дней, чтобы изготовить модель $B$.
- Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль $A$, и 7 дней —  автомобиль $B$.
- Каждый специалист может работать суммарно 30 дней.

Целевая функция будет выглядеть следующим образом:
    $$20000 A + 45000 B$$

Также запишем ограничения:
$$A, \ B \geq 0$$
$$4A + 5B \leq 30$$
$$3A +6B \leq 30$$
$$2A + 7B \leq 30$$

Заметьте, что здесь мы снова пишем обычные неравенства, а не условия в матричном виде. Дело в том, что для данной библиотеки так «удобнее», так как она принимает все условия в «первичном» виде.

In [140]:
# from pulp import *

# problem = LpProblem('Производство машин', LpMaximize)
# A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
# B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
# #Целевая функция
# problem += 20000*A + 45000*B 
# #Ограничения
# problem += 4*A + 5*B <= 30 
# problem += 3*A + 6*B <=30
# problem += 2*A + 7*B <=30
# problem.solve()
# print("Количество автомобилей модели А: ", A.varValue)
# print("Количество автомобилей модели В: ", B.varValue)
# print("Суммарный доход: ", value(problem.objective))

In [141]:
# problem.solve(PULP_CBC_CMD(msg=1))

#### Задание 6.1
Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:

![](https://lms.skillfactory.ru/assets/courseware/v1/4286673f4d92918f18bcf2ac7ee64863/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md6_6_1.png)

Сформулируйте предложенную задачу как задачу линейного программирования и решите её любым способом (желательно программным).

В качестве ответа введите минимальную суммарную стоимость поставки. Ответ округлите до целого числа.

In [143]:
from scipy.optimize import linprog
import numpy as np

cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])
stock = np.array([180, 220])
demand = np.array([110, 150, 140])
num_warehouse = 2
num_clients = 3

c = cost.flatten()

In [144]:
A = []
b = []
for i in range(0, num_warehouse):
    A.append([0] * (num_clients * i) + [1] * num_clients + [0] * (num_clients * (num_warehouse - i - 1)))
    b.append(stock[i])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[1 1 1 0 0 0]
 [0 0 0 1 1 1]]
[180 220]


In [145]:
A = A.tolist()
b = b.tolist()
for j in range(0, num_clients):
    A.append(([0] * j + [-1] + [0] * (num_clients - j - 1)) * num_warehouse)
    b.append(-demand[j])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[ 1  1  1  0  0  0]
 [ 0  0  0  1  1  1]
 [-1  0  0 -1  0  0]
 [ 0 -1  0  0 -1  0]
 [ 0  0 -1  0  0 -1]]
[ 180  220 -110 -150 -140]


In [146]:
ans = linprog(c=c, A_ub=A, b_ub=b)
(ans.x.round()*c).sum()

1900.0

##### Задание 6.2
В прошлом юните мы обсуждали задачу о назначениях исполнителей задач - теперь пришло время решить её.

Напомним суть: необходимо распределить пять задач между пятью исполнителями таким образом, чтобы суммарные затраты на работы были наименьшими.

![](https://lms.skillfactory.ru/assets/courseware/v1/3c57d0c14951762fe62a494509d15a69/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md6_6_2.png)

В качестве ответа введите минимальную стоимость работ.

In [1]:
from scipy.optimize import linprog
import numpy as np
import cvxpy as cvx
from itertools import product

In [2]:
c = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

In [3]:
x = cvx.Variable(shape=(5,5), boolean=True)
u = cvx.Variable(shape=5, integer=True)

In [4]:
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5),
    u >= 1,
    u <= 4
]

for i, j in product(range(5), range(5)):
    if i >= 1 and j >= 2 and i != j:
        constraints.append(u[i] - u[j] + 5 * x[i,j] <= 4)

In [5]:
func = cvx.sum(cvx.multiply(x, c))

In [6]:
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)

In [7]:
problem.solve()

32.0

# ===========================================

Метод золотого сечения
Определение:

Говорят, что точка x осуществляет золотое сечение отрезка $[a, \quad b]$, если

$\frac{b-a}{b-x}=\frac{b-x}{x-a}=\phi=\frac{1+\sqrt{5} }{2}$

В качестве $x_1$ и $x_2$ выберем точку золотого сечения отрезка и симметричную ей. Если $a<x_1<x_2<b$, то при указанном выборе точек получаем, что $x_1$ - точка золотого сечения отрезка $[a, \quad x_2]$, а $x_2$ - точка золотого сечения отрезка $[x_1, \quad b]$. Таким образом, на каждом шаге, кроме первого, необходимо вычислять значение только в одной точке, вторая берется из предыдущего шага.

Описание метода
Параметр на входе:  $\epsilon$ - достаточно малая положительная константа, погрешность метода.

1. $x_1 = b-\frac{b-a}{\phi}, \quad x_2 = a+\frac{b-a}{\phi}$

2. Повторять:

3. Если $f(x_1) > f(x_2)$, то $a=x_1$, $\quad x_1=x_2, \quad x_2=b-(x_1-a)$;
4. Если $f(x_1) < f(x_2)$, то $b=x_2$, $\quad x_2=x_1, \quad x_1=a+(b-x_2)$;
5. пока $\frac{b-a}{2} \geq \epsilon$;

6. $ \tilde{x}^{\ast}=\frac{a+b}{2}$.